In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# 加载环境变量
load_dotenv(dotenv_path='../.env')
api_key = os.getenv('OPENAI_API_KEY')
model_name = os.getenv('MODEL_NAME')
model_url = os.getenv('MODEL_URL')

# 初始化模型
model = ChatOpenAI(base_url=model_url, api_key=api_key, model=model_name)

# 定义工作流
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = model.invoke(state['messages'])
    return {'messages': [response]}

# 添加节点和边
workflow.add_node('model', call_model)
workflow.add_edge(START, 'model')

# 添加内存 checkpointer
memory = InMemorySaver()
app = workflow.compile(checkpointer=memory)

# 使用一个 thread_id 表示当前会话
config = {'configurable': {'thread_id': 'abc123'}}

print('=== 开始对话测试 ===')

# 对话 1
query = 'Hi! I am Bob.'
input_messages = [HumanMessage(query)]
output = app.invoke({'messages': input_messages}, config)
print('对话1 - 用户:', query)
print('对话1 - AI回复:')
output['messages'][-1].pretty_print()

# 对话 2（记忆测试）
query = 'What is my name?'
input_messages = [HumanMessage(query)]
output = app.invoke({'messages': input_messages}, config)
print('\\n对话2 - 用户:', query)
print('对话2 - AI回复:')
output['messages'][-1].pretty_print()

config2 = {'configurable': {'thread_id': 'def123'}}

# 对话 3（另外thread_id记忆测试）
query = 'What is my name?'
input_messages = [HumanMessage(query)]
output = app.invoke({'messages': input_messages}, config2)
print('\\n对话3 - 用户:', query)
print('对话3 - AI回复:')
output['messages'][-1].pretty_print()

=== 开始对话测试 ===
对话1 - 用户: Hi! I am Bob.
对话1 - AI回复:
================================== Ai Message ==================================

Hello, Bob! Nice to meet you. I'm Doubao, an AI developed by ByteDance.
\n对话2 - 用户: What is my name?
对话2 - AI回复:
================================== Ai Message ==================================

Your name is Bob, as you mentioned earlier! 😊
\n对话3 - 用户: What is my name?
对话3 - AI回复:
================================== Ai Message ==================================

I don’t know your name unless you tell me! You can share your name with me if you’d like.
